## CREDIT CARD DEFAULT PREDICTION AND MODEL INTERPRETATION                             USING POST- HOC EXPLAINABILITY TECHNIQUES 

### Synopysis

Objective – Predict the probability of a customer defaulting payment for the credit card the subsequent month, based on past information. The past information is provided in the dataset. This probability will help the collections team to prioritise follow up with customers who have a high propensity of defaulting. The goal of the research is to explore and exhibit the capabilities of various model agnostic interpretability techniques to generate explainable insights into prediction model outcomes

1. Getting the Data - https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients
2. Data Quality checks


* Input : Raw dataset from UCI (default_of_credit_card_clients.xls)
* Outcome : Cleaned dataset (inpt_df_processed.csv)

### Attributes description
This study uses 23 variables as explanatory variables, extracted/interpreted from : 
    
```
   ----------------------------------------------------------------------------------------
   Feature Name         Feature Description        
   -------------------- -------------------------------------------------------------------
   limit_bal            Amount of the given credit (NT dollar): it includes both the individual 
                        consumer credit and his/her family (supplementary) credit.
                        
   sex                  Gender (1 = male; 2 = female)
   
   education            Education (1 = graduate school; 2 = university; 3 = high school; 4 = others)
   
   marriage             Marital status (1 = married; 2 = single; 3 = others)
   
   age                  Age (in years)
   
   pay_1 - pay_6        History of past payment. Past monthly payment records 
                        From April to September, 2005 as follows:                     
                        pay_1 = the repayment status in September, 2005
                        pay_2 = the repayment status in August, 2005
                        ...
                        pay_6 = the repayment status in April, 2005 
                        
                        The measurement scale for the repayment status is: 
                        -1 = pay duly; 
                        1 = payment delay for one month 
                        2 = payment delay for two months
                        ...
                        8 = payment delay for eight months 
                        9 = payment delay for nine months and above
                        
   bill_amt1-bill_amt5  Amount of bill statement (NT dollar). 
                        bill_amt1 = amount of bill statement in September, 2005 
                        bill_amt2 = amount of bill statement in August, 2005
                        ...
                        bill_amt6= amount of bill statement in April, 2005 
                        
   pay_amt1-pay_amt6    Amount of previous payment (NT dollar)
                        pay_amt1 = amount paid in September, 2005
                        pay_amt2 = amount paid in August, 2005
                        ...
                        pay_amt6 = amount paid in April, 2005 
   ----------------------------------------------------------------------------------------
```

In [1]:
# Import libraries
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Import plotting libraries
import seaborn as sns
import matplotlib.pyplot as plt
import pylab

#### 2. Data Importing

In [3]:
#--------------------------------------------------------------
#Importing the excel file as python dataframe
#--------------------------------------------------------------

inpt_df=pd.read_excel('default_of_credit_card_clients.xls',header=1)
inpt_df.head(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
5,6,50000,1,1,2,37,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0
6,7,500000,1,1,2,29,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
7,8,100000,2,2,2,23,0,-1,-1,0,...,221,-159,567,380,601,0,581,1687,1542,0
8,9,140000,2,3,1,28,0,0,2,0,...,12211,11793,3719,3329,0,432,1000,1000,1000,0
9,10,20000,1,3,2,35,-2,-2,-2,-2,...,0,13007,13912,0,0,0,13007,1122,0,0


In [4]:
#--------------------------------------------------------------
#Size and shape of the dataset - There are 30,000 observations and 25 columns
#--------------------------------------------------------------

print("Shape of data:", inpt_df.shape)

print("Numerical features count: ", len(inpt_df.dtypes[inpt_df.dtypes != "object"].index))

print("Categorical features count: ", len(inpt_df.dtypes[inpt_df.dtypes == "object"].index))

Shape of data: (30000, 25)
Numerical features count:  25
Categorical features count:  0


#### 2. Data Quality checks

In [5]:
#--------------------------------------------------------------
#1. Check for missing/null values - There are no missing values
#--------------------------------------------------------------

inpt_df.isnull().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64

In [8]:
#--------------------------------------------------------------
#2. Check for duplicates - No duplicate rows found
#--------------------------------------------------------------

print("No of unique records: ", inpt_df['ID'].nunique())

No of unique records:  30000


In [9]:
#--------------------------------------------------------------
#3. Elimination of unnecessary variable and data Cleaning
#--------------------------------------------------------------

# Dropping unnecessary feature(ID) which do not contribute significantly
inpt_df = inpt_df.drop(["ID"],axis=1)

#Renaming Pay_0 to Pay_1 to correct the numbering order of payment status
inpt_df.rename(columns={'PAY_0':'PAY_1'}, inplace=True)
inpt_df.rename(columns={'default payment next month':'DEFAULT'}, inplace=True)

In [10]:
#--------------------------------------------------------------
#4. Data Correctness check 
#--------------------------------------------------------------

print(inpt_df.AGE[inpt_df.AGE<18].count(),
      inpt_df.LIMIT_BAL[inpt_df.LIMIT_BAL<0].count(),
      inpt_df.DEFAULT[inpt_df.DEFAULT>1].count())

0 0 0


In [8]:
#4.A Distribution of categorical variables: EDUCATION, MARRIAGE and SEX

    #There is no description available for 0 in Education and Marriage fields
    #EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
    #MARRIAGE: (1=married, 2=single, 3=others) 

print(inpt_df['EDUCATION'].value_counts())
print(inpt_df['MARRIAGE'].value_counts())
print(inpt_df['SEX'].value_counts())

2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: EDUCATION, dtype: int64
2    15964
1    13659
3      323
0       54
Name: MARRIAGE, dtype: int64
2    18112
1    11888
Name: SEX, dtype: int64


In [9]:
#4.B Replacing all unknown class values as one category 'Others'

inpt_df["EDUCATION"]=inpt_df["EDUCATION"].map({0:4,1:1,2:2,3:3,4:4,5:4,6:4})
inpt_df["MARRIAGE"]=inpt_df["MARRIAGE"].map({0:3,1:1,2:2,3:3})

In [10]:
#4.C Payment status for 6 months - There is no description for -2 and 0

    #As per dataset creator's response in https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset/discussion/34608
    # -2 : No consumption/transaction; -1 : Paid in full; 0 : The use of revolving credit; 
    
inpt_df[['PAY_1', 'PAY_2', 'PAY_3','PAY_4','PAY_5','PAY_6']].apply(pd.Series.value_counts)

,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
-2,2759,3782,4085,4348,4546.0,4895.0
-1,5686,6050,5938,5687,5539.0,5740.0
0,14737,15730,15764,16455,16947.0,16286.0
1,3688,28,4,2,NaN,NaN
2,2667,3927,3819,3159,2626.0,2766.0
3,322,326,240,180,178.0,184.0
4,76,99,76,69,84.0,49.0
5,26,25,21,35,17.0,13.0
6,11,12,23,5,4.0,19.0
7,9,20,27,58,58.0,46.0


In [11]:
#4.D Standardizing the values of payment status field by marking -2,-1 and 0 as 0

repl = (inpt_df.PAY_1 == -2) | (inpt_df.PAY_1 == -1) | (inpt_df.PAY_1 == 0)
inpt_df.loc[repl, 'PAY_1'] = 0
repl = (inpt_df.PAY_2 == -2) | (inpt_df.PAY_2 == -1) | (inpt_df.PAY_2 == 0)
inpt_df.loc[repl, 'PAY_2'] = 0
repl = (inpt_df.PAY_3 == -2) | (inpt_df.PAY_3 == -1) | (inpt_df.PAY_3 == 0)
inpt_df.loc[repl, 'PAY_3'] = 0
repl = (inpt_df.PAY_4 == -2) | (inpt_df.PAY_4 == -1) | (inpt_df.PAY_4 == 0)
inpt_df.loc[repl, 'PAY_4'] = 0
repl = (inpt_df.PAY_5 == -2) | (inpt_df.PAY_5 == -1) | (inpt_df.PAY_5 == 0)
inpt_df.loc[repl, 'PAY_5'] = 0
repl = (inpt_df.PAY_6 == -2) | (inpt_df.PAY_6 == -1) | (inpt_df.PAY_6 == 0)
inpt_df.loc[repl, 'PAY_6'] = 0

In [24]:
# Saving cleansed file in csv format for part 2 - EDA

inpt_df.to_csv("inpt_df_processed.csv", index=None)